# Loan Payback Prediction - Kaggle Competition

An ensemble machine learning solution using LightGBM and XGBoost to predict loan repayment probability.

## 📊 Overview
- **Objective**: Predict whether a borrower will repay their loan
- **Metric**: ROC AUC Score
- **Approach**: Feature engineering + Ensemble modeling (LightGBM + XGBoost)

## 🎯 Features
- 12 engineered features (financial ratios, categories, interactions)
- Stratified validation split
- Early stopping to prevent overfitting
- Model averaging for robust predictions

## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb

# Visualization (optional - can be commented out)
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

## 2. Load Data

In [ ]:
train = pd.read_csv('/kaggle/input/playground-series-s5e11/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s5e11/test.csv')

print(f"Training data: {train.shape}, Test data: {test.shape}")

### 2.1 Quick EDA - Key Insights (Optional)

In [ ]:
# Optional: Visualize key insights (comment out if not needed)
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# 1. Feature Correlation Heatmap - shows which features are most important
numerical_features = ['annual_income', 'debt_to_income_ratio', 'credit_score', 
                      'loan_amount', 'interest_rate', 'loan_paid_back']
correlation_matrix = train[numerical_features].corr()

sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, fmt='.2f', ax=axes[0], 
            cbar_kws={'label': 'Correlation Coefficient'})
axes[0].set_title('Feature Correlation Matrix\n(Key: Credit Score & Interest Rate)', 
                  fontsize=13, fontweight='bold', pad=15)

# 2. Target Distribution by Credit Score - business insight
credit_bins = pd.cut(train['credit_score'], bins=[0, 580, 670, 740, 800, 850],
                     labels=['Poor\n(<580)', 'Fair\n(580-670)', 'Good\n(670-740)', 
                             'Very Good\n(740-800)', 'Excellent\n(800+)'])
payback_rate = train.groupby(credit_bins)['loan_paid_back'].agg(['mean', 'count'])

bars = axes[1].bar(range(len(payback_rate)), payback_rate['mean'], 
                   color=['#d62728', '#ff7f0e', '#ffcc00', '#2ca02c', '#1f77b4'], 
                   edgecolor='black', alpha=0.8)
axes[1].set_xticks(range(len(payback_rate)))
axes[1].set_xticklabels(payback_rate.index)
axes[1].set_ylabel('Loan Payback Rate', fontsize=11, fontweight='bold')
axes[1].set_xlabel('Credit Score Category', fontsize=11, fontweight='bold')
axes[1].set_title('Loan Payback Rate by Credit Score\n(Clear trend: Higher score = Higher payback)', 
                  fontsize=13, fontweight='bold', pad=15)
axes[1].set_ylim(0, 1)
axes[1].grid(axis='y', alpha=0.3)

# Add percentage labels on bars
for i, (bar, rate) in enumerate(zip(bars, payback_rate['mean'])):
    height = bar.get_height()
    axes[1].text(bar.get_x() + bar.get_width()/2., height + 0.02,
                f'{rate:.1%}\n(n={payback_rate["count"].iloc[i]:,})',
                ha='center', va='bottom', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\n{'='*60}")
print("KEY INSIGHTS FROM EDA:")
print(f"{'='*60}")
print(f"✓ Credit Score shows STRONGEST correlation with loan payback")
print(f"✓ Interest Rate is negatively correlated (higher rate = riskier loans)")
print(f"✓ Excellent credit (800+) has {payback_rate['mean'].iloc[-1]:.1%} payback rate")
print(f"✓ Poor credit (<580) has {payback_rate['mean'].iloc[0]:.1%} payback rate")
print(f"✓ This validates our feature engineering approach!")
print(f"{'='*60}\n")

## 3. Feature Engineering

In [ ]:
def create_features(df):
    """
    Engineer financial and categorical features to improve model performance.
    
    Creates 12 new features:
    - Financial ratios: loan/income, payment/income, debt/income
    - Monthly payment estimation based on interest rate
    - Categorical bins: credit score, income, interest rate levels
    - Grade decomposition: separate grade and subgrade
    - Interaction: income × credit score
    - Custom risk score composite
    
    Args:
        df: DataFrame with loan application data
        
    Returns:
        DataFrame with original + engineered features
    """
    df = df.copy()
    
    # Financial ratios
    df['loan_to_income_ratio'] = df['loan_amount'] / (df['annual_income'] + 1)
    df['estimated_monthly_payment'] = (df['loan_amount'] * (df['interest_rate'] / 100) / 12) / (1 - (1 + df['interest_rate'] / 100 / 12) ** -60)
    df['payment_to_income_ratio'] = (df['estimated_monthly_payment'] * 12) / (df['annual_income'] + 1)
    df['total_debt'] = df['annual_income'] * df['debt_to_income_ratio']
    df['total_debt_with_loan'] = df['total_debt'] + df['loan_amount']
    df['new_debt_to_income_ratio'] = df['total_debt_with_loan'] / (df['annual_income'] + 1)
    
    # Categorical features
    df['credit_score_category'] = pd.cut(df['credit_score'], bins=[0, 580, 670, 740, 800, 850], labels=['Poor', 'Fair', 'Good', 'Very Good', 'Excellent'])
    df['income_category'] = pd.cut(df['annual_income'], bins=[0, 30000, 50000, 75000, 100000, np.inf], labels=['Low', 'Medium-Low', 'Medium', 'Medium-High', 'High'])
    df['interest_rate_category'] = pd.cut(df['interest_rate'], bins=[0, 7, 11, 15, np.inf], labels=['Low', 'Medium', 'High', 'Very High'])
    
    # Grade decomposition
    df['grade'] = df['grade_subgrade'].str[0]
    df['subgrade'] = df['grade_subgrade'].str[1:]
    
    # Interaction and composite features
    df['income_credit_interaction'] = df['annual_income'] * df['credit_score']
    df['risk_score'] = (df['debt_to_income_ratio'] * 0.3 + df['interest_rate'] * 0.2 - df['credit_score'] / 1000 * 0.5)
    
    return df

train_fe = create_features(train)
test_fe = create_features(test)

## 4. Data Preprocessing

In [ ]:
def preprocess_data(train_df, test_df):
    """
    Preprocess data: encode categoricals, scale numericals, handle missing values.
    
    Steps:
    1. Separate features and target
    2. Label encode all categorical variables
    3. Fill missing values with median
    4. Standard scale numerical features
    
    Args:
        train_df: Training DataFrame with features and target
        test_df: Test DataFrame with features only
        
    Returns:
        X_train, X_test, y_train: Processed features and target
    """
    X_train = train_df.drop(['id', 'loan_paid_back'], axis=1)
    y_train = train_df['loan_paid_back']
    X_test = test_df.drop(['id'], axis=1)
    
    categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
    numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
    
    # Label encode categorical features (fit on combined data for consistency)
    for col in categorical_cols:
        le = LabelEncoder()
        combined = pd.concat([X_train[col], X_test[col]], axis=0)
        le.fit(combined.astype(str))
        X_train[col] = le.transform(X_train[col].astype(str))
        X_test[col] = le.transform(X_test[col].astype(str))
    
    # Handle missing values
    X_train = X_train.fillna(X_train.median())
    X_test = X_test.fillna(X_test.median())
    
    # Scale numerical features
    scaler = StandardScaler()
    X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
    X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])
    
    return X_train, X_test, y_train

X_train, X_test, y_train = preprocess_data(train_fe, test_fe)

## 5. Model Training

In [ ]:
# Split for validation
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train, y_train, test_size=0.2, random_state=RANDOM_STATE, stratify=y_train
)

In [ ]:
# Train LightGBM
lgb_params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'max_depth': -1,
    'min_child_samples': 20,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'random_state': RANDOM_STATE,
    'n_jobs': -1,
    'verbose': -1
}

lgb_train = lgb.Dataset(X_train_split, y_train_split)
lgb_val = lgb.Dataset(X_val_split, y_val_split, reference=lgb_train)

lgb_model = lgb.train(
    lgb_params,
    lgb_train,
    num_boost_round=1000,
    valid_sets=[lgb_val],
    callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(period=0)]
)

y_val_pred_lgb = lgb_model.predict(X_val_split, num_iteration=lgb_model.best_iteration)
lgb_auc = roc_auc_score(y_val_split, y_val_pred_lgb)
print(f"LightGBM AUC: {lgb_auc:.5f}")

In [ ]:
# Train XGBoost
xgb_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'max_depth': 6,
    'learning_rate': 0.05,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'min_child_weight': 3,
    'reg_alpha': 0.1,
    'reg_lambda': 1,
    'random_state': RANDOM_STATE,
    'n_jobs': -1
}

xgb_train = xgb.DMatrix(X_train_split, label=y_train_split)
xgb_valid = xgb.DMatrix(X_val_split, label=y_val_split)

xgb_model = xgb.train(
    xgb_params,
    xgb_train,
    num_boost_round=1000,
    evals=[(xgb_valid, 'valid')],
    early_stopping_rounds=50,
    verbose_eval=0
)

y_val_pred_xgb = xgb_model.predict(xgb.DMatrix(X_val_split))
xgb_auc = roc_auc_score(y_val_split, y_val_pred_xgb)
print(f"XGBoost AUC: {xgb_auc:.5f}")

In [ ]:
# Ensemble
y_val_pred_ensemble = (y_val_pred_lgb + y_val_pred_xgb) / 2
ensemble_auc = roc_auc_score(y_val_split, y_val_pred_ensemble)
print(f"Ensemble AUC: {ensemble_auc:.5f}")

## 6. Train Final Models on Full Data

In [ ]:
# Retrain on full data
lgb_train_full = lgb.Dataset(X_train, y_train)
lgb_model_final = lgb.train(
    lgb_params,
    lgb_train_full,
    num_boost_round=lgb_model.best_iteration,
    callbacks=[lgb.log_evaluation(period=0)]
)

xgb_train_full = xgb.DMatrix(X_train, label=y_train)
xgb_model_final = xgb.train(
    xgb_params,
    xgb_train_full,
    num_boost_round=xgb_model.best_iteration,
    verbose_eval=0
)

print("Models trained on full data")

## 7. Generate Predictions and Save

In [ ]:
# Generate predictions
test_pred_lgb = lgb_model_final.predict(X_test, num_iteration=lgb_model_final.best_iteration)
test_pred_xgb = xgb_model_final.predict(xgb.DMatrix(X_test))
test_pred_ensemble = (test_pred_lgb + test_pred_xgb) / 2

# Create submission
submission = pd.DataFrame({
    'id': test_fe['id'],
    'loan_paid_back': test_pred_ensemble
})

submission.to_csv('submission.csv', index=False)
print(f"Submission saved: {submission.shape}")

## 📝 Summary

### Model Architecture
- **LightGBM**: Gradient boosting with early stopping (50 rounds)
- **XGBoost**: Extreme gradient boosting with similar hyperparameters
- **Ensemble**: Simple averaging for improved generalization

### Key Hyperparameters
- Learning rate: 0.05 (both models)
- Early stopping: 50 rounds
- Max iterations: 1000
- Validation split: 20% stratified

### Feature Engineering Impact
Created 12 engineered features that capture:
- Financial health indicators (debt ratios, payment capacity)
- Risk categorization (credit score tiers, income levels)
- Interaction effects (income × credit score)

### Next Steps for Improvement
1. Hyperparameter tuning with Optuna/GridSearchCV
2. Add CatBoost to ensemble
3. Implement K-fold cross-validation
4. Try stacking/blending instead of averaging
5. Feature selection to reduce overfitting

---
**Author**: Your Name | **Date**: 2025 | **Competition**: Kaggle Playground Series S5E11